<a href="https://colab.research.google.com/github/ParalelaUnsaac/G3-2020-1/blob/main/Paralela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!nvidia-smi


Mon Oct 26 22:57:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%cu
#include "cuda_runtime.h"  // Definicion de tipo de integrados y superposiciones de funciones para las extensiones del lenguaje CUDA
#include <stdio.h> // cabezera input-output
#include <stdlib.h> //biblioteca estandar
#include <cuda.h> //define el host publico funciones y tipos para la API de controlador CUDA

#define MAXVALUE 10000

//------------------------------------

void numberGen(int N, int max, int* store) //Funcion Generar numeros
{
    int i;
    srand(time(0)); //tiempo para generar numeros aleatorios
    for (i = 0; i < N; i++)
        store[i] = rand() % max; //Almacena en el arreglo
}
//------------------------------------

__global__ void countOdds(int* d, int N, int* odds)
{
    #asignamos memoria compartida dinamicamente(extern significa quela matriz hace referencia a la memoria compartida declarada en otro lugar)
    extern __shared__ int count[];

    int myID = blockIdx.x * blockDim.x + threadIdx.x; #ñadiendo el tamaño del grid y del bloque 
    int localID = threadIdx.x; #ponemos el id en cada hilo
    count[localID] = 0;
    if (myID < N)
        count[localID] = (d[myID] % 2);
    __syncthreads();

    // fase de reducción 
    int step = 1;
    while (((localID | step) < blockDim.x) && ((localID & step) == 0))
    {
        count[localID] += count[localID | step];
        step <<= 1;
        __syncthreads();
    }

   
    // adiciona al contador global
    if (localID == 0)
        atomicAdd(odds, count[0]);
}

//------------------------------------
#Generar Tamaño compartido
int sharedSize(int b)
{
    #Devuelve la cantidad de memoria asignada
    return b * sizeof(int);
}
//------------------------------------

int main(int argc, char** argv) //Funcion principal
{
    #Definimos la cantidad de numeros que se generaran aleatoriamente
    int N = 2000; 
    #Define punteros host (h*) y device (d*)
    int* ha, * hres, * da, * dres;   

    ha = new int[N]; #declaramos un arreglo en la variable ha
    hres = new int[1];#declaramos un arreglo en la variable hres

    cudaMalloc((void**)&da, sizeof(int) * N);
    cudaMalloc((void**)&dres, sizeof(int) * 1);

    #Llama a la funcion numberGen()
    numberGen(N, MAXVALUE, ha);

    cudaMemcpy(da, ha, sizeof(int) * N, cudaMemcpyHostToDevice); # copia datos a la memoria del dispositivo
    cudaMemset(dres, 0, sizeof(int));# #llenamos los primeros bytes de recuento del área de memoria apuntada por dres con el valor de byte valor constante.

    int blockSize, gridSize;
 #Devuelve el tamaño de cuadrícula y bloque que logra la máxima ocupación potencial para una función de dispositivo.
    cudaOccupancyMaxPotentialBlockSizeVariableSMem(&gridSize, &blockSize, (void*)countOdds, sharedSize, N);

    gridSize = ceil(1.0 * N / blockSize);
    printf("Grid : %i    Block : %i\n", gridSize, blockSize);
    countOdds << < gridSize, blockSize, blockSize * sizeof(int) >> > (da, N, dres);

    # copia datos a la memoria del dispositivo
    cudaMemcpy(hres, dres, sizeof(int), cudaMemcpyDeviceToHost);
    // chequeo
    int oc = 0;
    for (int i = 0; i < N; i++)
        if (ha[i] % 2){
            printf("%i ",ha[i]);
            oc++;
        }
    printf("\n");
    #imprime cantidad de numeros impares en el arreglo        
    printf("Cantidad de numeros impares - Chequeo\n");
    printf("%i %i\n", *hres, oc);

    cudaFree((void*)da); #Libera memoria de da
    cudaFree((void*)dres);#Libera memoria de dres
    delete[]ha;
    delete[]hres;
    cudaDeviceReset(); #Libera contexto de cuda , quita las asiganciones de todo los dispositivos
    getchar();
    return 0;
}

/tmp/tmp5o9c4akp/21472ac2-addf-465f-a861-27d2fe378277.cu:21:6: error: invalid preprocessing directive #asignamos
     #asignamos memoria compartida dinamicamente(extern significa quela matriz hace referencia a la memoria compartida declarada en otro lugar)
      ^~~~~~~~~
/tmp/tmp5o9c4akp/21472ac2-addf-465f-a861-27d2fe378277.cu:71:3: error: invalid preprocessing directive #Devuelve
  #Devuelve el tamaño de cuadrícula y bloque que logra la máxima ocupación potencial para una función de dispositivo.
   ^~~~~~~~

